In [43]:
! pip install kafka-python tensorflow gym numpy pandas


  Using cached kafka_python-2.1.1-py2.py3-none-any.whl.metadata (9.1 kB)
ERROR: Could not find a version that satisfies the requirement tensorflow (from versions: none)
ERROR: No matching distribution found for tensorflow


In [36]:
! pip install stable-baselines3==2.4.0 kafka-python pandas matplotlib


  Using cached stable_baselines3-2.4.0-py3-none-any.whl.metadata (4.5 kB)
  Using cached kafka_python-2.1.1-py2.py3-none-any.whl.metadata (9.1 kB)
  Using cached gymnasium-1.0.0-py3-none-any.whl.metadata (9.5 kB)
  Using cached numpy-1.26.4.tar.gz (15.8 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of stable-baselines3 to determine which version is compatible with other requirements. This could take a while.
ERROR: Could not find a version that satisfies the requirement torch>=1.13 (from stable-baselines3) (from versions: none)
ERROR: No matching distribution found for torch>=1.13


In [44]:
import numpy as np
import gym
from gym import spaces

class CarbonEmissionsEnv(gym.Env):
    def __init__(self, kafka_consumer):
        super(CarbonEmissionsEnv, self).__init__()
        self.kafka_consumer = kafka_consumer
        
        # Define action and observation space
        self.action_space = spaces.Discrete(3)  # Actions: 0 - Reduce coal, 1 - Promote renewables, 2 - Recommend policy
        self.observation_space = spaces.Box(low=0, high=np.inf, shape=(4,), dtype=np.float32)  # Example: Country, Year, Carbon, Activity
        
        # Initialize state
        self.state = None
        self.done = False
    
    def step(self, action):
        # Apply the chosen action and calculate reward
        reward = 0
        if action == 0:  # Reduce coal
            reward = -self.state[2] * 0.1
        elif action == 1:  # Promote renewables
            reward = -self.state[3] * 0.2
        elif action == 2:  # Recommend policy
            reward = -self.state[2] * 0.05
        
        # Simulate new state from Kafka
        self.state = self._get_new_state()
        
        # Check if episode is done
        self.done = self.state is None
        
        return self.state, reward, self.done, {}
    
    def reset(self):
        # Reset state using Kafka data
        self.state = self._get_new_state()
        return self.state
    
    def _get_new_state(self):
        # Fetch new data from Kafka
        for message in self.kafka_consumer:
            data = json.loads(message.value)
            return np.array([data['country'], data['year'], data['carbon_emission'], data['activity']])
        return None


In [4]:
! pip install gym

  Using cached gym-0.26.2.tar.gz (721 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached cloudpickle-3.1.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached gym_notices-0.0.8-py3-none-any.whl.metadata (1.0 kB)
Using cached cloudpickle-3.1.1-py3-none-any.whl (20 kB)
Using cached gym_notices-0.0.8-py3-none-any.whl (3.0 kB)
  Created wheel for gym: filename=gym-0.26.2-py3-none-any.whl size=827694 sha256=df2812a95d2aa3c545752693112972bf17543cc586af3be6dd48884f385d0499
  Stored in directory: /Users/kdn_aikothalavanya/Library/Caches/pip/wheels/1d/34/c6/856a1e1eff47d8f18545c833b6138ae1e9f53c7de9bcc5f31d
Successfully built gym

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [6]:
import gym
from gym import spaces
import numpy as np

class CarbonEmissionEnv(gym.Env):
    def __init__(self, data):
        super(CarbonEmissionEnv, self).__init__()
        self.data = data  # List of data points
        self.current_step = 0

        # Define action and observation space
        self.action_space = spaces.Discrete(3)  # Actions: Reduce, Maintain, Increase emissions
        self.observation_space = spaces.Box(
            low=0, high=np.inf, shape=(3,), dtype=np.float32
        )  # Features: coal, year, total emissions

    def reset(self):
        self.current_step = 0
        return self._next_observation()

    def _next_observation(self):
        row = self.data[self.current_step]
        return np.array([row["coal"], row["year"], row["total"]])

    def step(self, action):
        row = self.data[self.current_step]
        reward = self._calculate_reward(action, row)

        self.current_step += 1
        done = self.current_step >= len(self.data)

        return self._next_observation(), reward, done, {}

    def _calculate_reward(self, action, row):
        coal_emission = row["coal"]
        if action == 0:  # Reduce emissions
            reward = -coal_emission * 0.1
        elif action == 1:  # Maintain emissions
            reward = -coal_emission * 0.05
        else:  # Increase emissions
            reward = -coal_emission * 0.2
        return reward


In [19]:
! pip install --upgrade pip


  Using cached pip-25.0.1-py3-none-any.whl.metadata (3.7 kB)
Using cached pip-25.0.1-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 25.0
    Uninstalling pip-25.0:
      Successfully uninstalled pip-25.0


In [29]:
! pip install stable-baselines3==2.5.0 gymnasium==0.29.1 



  Using cached stable_baselines3-2.5.0-py3-none-any.whl.metadata (4.8 kB)
  Using cached gymnasium-0.29.1-py3-none-any.whl.metadata (10 kB)
INFO: pip is looking at multiple versions of stable-baselines3 to determine which version is compatible with other requirements. This could take a while.
ERROR: Could not find a version that satisfies the requirement torch<3.0,>=2.3 (from stable-baselines3) (from versions: none)
ERROR: No matching distribution found for torch<3.0,>=2.3


In [37]:
! pip install stable-baselines3
! pip install PPO


  Using cached stable_baselines3-2.5.0-py3-none-any.whl.metadata (4.8 kB)
  Using cached gymnasium-1.0.0-py3-none-any.whl.metadata (9.5 kB)
INFO: pip is looking at multiple versions of stable-baselines3 to determine which version is compatible with other requirements. This could take a while.
  Using cached stable_baselines3-2.4.1-py3-none-any.whl.metadata (4.5 kB)
  Using cached numpy-1.26.4.tar.gz (15.8 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached stable_baselines3-2.4.0-py3-none-any.whl.metadata (4.5 kB)
  Using cached stable_baselines3-2.3.2-py3-none-any.whl.metadata (5.1 kB)
  Using cached gymnasium-0.29.1-py3-none-any.whl.metadata (10 kB)
  Using cached stable_baselines3-2.2.1-py3-none-any.whl.metadata (5.0 kB)
  Using cached stable_baselines3-2.1.0-py3-none-any.whl.metadata (5.2 kB)
  Using cached stable_baselines3-2.0.0-py3-none-any.whl.metadata (5.4 kB)
  Using cached g

In [ ]:
import tensorflow as tf 
from tensorflow.keras import models, layers
from collections import deque
import random

class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95  # Discount factor
        self.epsilon = 1.0  # Exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()
    
    def _build_model(self):
        model = models.Sequential([
            layers.Dense(24, input_dim=self.state_size, activation='relu'),
            layers.Dense(24, activation='relu'),
            layers.Dense(self.action_size, activation='linear')
        ])
        model.compile(optimizer=tf.keras.optimizers.Adam(lr=self.learning_rate), loss='mse')
        return model
    
    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        q_values = self.model.predict(state)
        return np.argmax(q_values[0])
    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
    
    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target += self.gamma * np.amax(self.model.predict(next_state)[0])
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

# Initialize environment and agent
state_size = 4
action_size = 3
env = CarbonEmissionsEnv(consumer)
agent = DQNAgent(state_size, action_size)

# Train the agent
episodes = 100
for e in range(episodes):
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    for time in range(500):
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        next_state = np.reshape(next_state, [1, state_size])
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        if done:
            print(f"episode: {e}/{episodes}, score: {time}")
            break
    agent.replay(32)


ModuleNotFoundError: No module named 'tensorflow'